# Analyse 3

We vragen ons af of er meer doelpunten worden gemaakt wanneer 2 ploegen die ver uit elkaar staan in het klassement tegen elkaar spelen, dan in het geval de 2 ploegen dicht bij elkaar staan in het klassement. 
Verdeel voor elk seizoen het klassement denkbeeldig in 3 delen: 
- (1) de eerste 6 ploegen
- (2) de laatste 6 ploegen
- (3) alle ploegen ertussen (eventueel meer of minder dan 6)

Breid de dataframe wedstrijden uit met de positie van de thuisploeg en de uitploeg in het klassement (van de voorgaande speeldag!).
Breid de dataframe wedstrijden uit met het denkbeeldige deel van de thuisploeg en de uitploeg in het klassement: (1), (2) of (3)
Breid de dataframe wedstrijden uit met de categorie waartoe de wedstrijd behoort
- dicht: beide ploegen behoren tot hetzelfde denkbeeldige deel van het klassement (1), (2) of (3)
- gemiddeld: 1 ploeg behoort tot (1) of (2), de andere ploeg behoort tot (3)
- ver: 1 ploeg behoort tot (1) en de andere tot (2)

Bereken over alle seizoenen heen het totaal aantal wedstrijden én het totaal aantal doelpunten voor de 3 categorieën (dicht / gemiddeld / ver). Maak gebruik van wat aan bod komt in Data Science & AI Hoofdstuk 4 (Goodness of fit test) en wat je hiervoor berekend hebt om te berekenen of er significant meer gescoord wordt in de categorie ver dan in de categorie dicht

In [14]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns                               # Advanced data visualisation

from scipy.stats import chi2_contingency            # Chi-square test

In [15]:
# Datasets inladen
wedstrijdendata = pd.read_csv("D:/Hogent/Visual Studio Code/DEP/DEP1-2023-2024-groep30/transfermarkt/data/correcte_data/wedstrijden.csv", sep=";")
klassementdata = pd.read_csv("D:/Hogent/Visual Studio Code/DEP/DEP1-2023-2024-groep30/transfermarkt/data/correcte_data/klassement.csv", sep=";")

In [16]:
# De eerste 10 rijen van de dataset bekijken
wedstrijdendata.head(10)

,Seizoen,Speeldag,Datum,Tijdstip,Id,StamnummerThuisploeg,RoepnaamThuisploeg,StamnummerUitploeg,RoepnaamUitploeg,StandThuis,StandUit
0,1960,1,1960-09-04,15:00,4008203,373,Sint-Truidense VV,16,Standard Luik,0,2
1,1960,1,1960-09-04,15:00,4008204,90,Eendracht Aalst,1,Royal Antwerp FC,3,2
2,1960,1,1960-09-04,15:00,4008205,246,Olympic Charleroi,35,RSC Anderlecht,0,7
3,1960,1,1960-09-04,15:00,4008206,13,Beerschot,33,Verviétois,1,0
4,1960,1,1960-09-04,15:00,4008207,3,Club Brugge,10,Union Saint-Gilloise,1,2
5,1960,1,1960-09-04,15:00,4008208,2,Daring Club Brussel,3434,Patro Eisden,2,1
6,1960,1,1960-09-04,15:00,4008209,4,RFC Luik,7,KAA Gent,2,1
7,1960,1,1960-09-04,15:00,4008210,553,Waterschei SV Thor,30,Lierse SK,5,0
8,1960,2,1960-09-11,15:00,4008211,16,Standard Luik,90,Eendracht Aalst,3,0
9,1960,2,1960-09-11,15:00,4008212,35,RSC Anderlecht,4,RFC Luik,2,3


In [17]:
# De eerste 10 rijen van de dataset bekijken
klassementdata.head(10)

,Seizoen,Speeldag,Stand,Stamnummer,Ploeg,AantalGespeeld,AantalGewonnen,AantalGelijk,AantalVerloren,DoelpuntenVoor,DoelpuntenTegen,Doelpuntensaldo,Links_Tweepuntensysteem,Rechts_Tweepuntensysteem,Driepuntensysteem
0,1960,1,1,35,RSC Anderlecht,1,1,0,0,7,0,7,2,0,3
1,1960,1,2,553,Waterschei SV Thor,1,1,0,0,5,0,5,2,0,3
2,1960,1,3,16,Standard Luik,1,1,0,0,2,0,2,2,0,3
3,1960,1,4,90,Eendracht Aalst,1,1,0,0,3,2,1,2,0,3
4,1960,1,5,2,Daring Club Brussel,1,1,0,0,2,1,1,2,0,3
5,1960,1,5,4,RFC Luik,1,1,0,0,2,1,1,2,0,3
6,1960,1,5,10,Union Saint-Gilloise,1,1,0,0,2,1,1,2,0,3
7,1960,1,8,13,Beerschot,1,1,0,0,1,0,1,2,0,3
8,1960,1,9,1,Royal Antwerp FC,1,0,0,1,2,3,-1,0,2,0
9,1960,1,10,3,Club Brugge,1,0,0,1,1,2,-1,0,2,0


In [22]:
# Functie om de posities van teams toe te voegen aan het wedstrijden DataFrame
def voeg_posities_toe(wedstrijden, klassement):
    # Zet 'Seizoen' in beide dataframes om naar strings
    wedstrijden['Seizoen'] = wedstrijden['Seizoen'].astype(str)
    klassement['Seizoen'] = klassement['Seizoen'].astype(str)
    
    # Merge op 'Seizoen', 'Speeldag', en 'Stamnummer' voor thuisploeg
    wedstrijden = pd.merge(wedstrijden, klassement, left_on=['Seizoen', 'Speeldag', 'StamnummerThuisploeg'], right_on=['Seizoen', 'Speeldag', 'Stamnummer'], how='left')
    wedstrijden.rename(columns={'Stand': 'PositieThuis'}, inplace=True)
    
    # Merge op 'Seizoen', 'Speeldag', en 'Stamnummer' voor uitploeg
    wedstrijden = pd.merge(wedstrijden, klassement, left_on=['Seizoen', 'Speeldag', 'StamnummerUitploeg'], right_on=['Seizoen', 'Speeldag', 'Stamnummer'], how='left')
    wedstrijden.rename(columns={'Stand': 'PositieUit'}, inplace=True)
    
    return wedstrijden


# Functie om segmenten te bepalen op basis van posities
def segment_bepalen(positie, totaal_teams):
    if positie <= 6:
        return 1  # Top 6 teams
    elif positie > totaal_teams - 6:
        return 2  # Laatste 6 teams
    else:
        return 3  # Midden teams


# Dataframe uitbreiden met segment en categorie
def categoriseer_wedstrijden(wedstrijden):
    # Eerst segmenten toevoegen gebaseerd op posities
    wedstrijden['SegmentThuis'] = wedstrijden['PositieThuis'].apply(lambda x: segment_bepalen(x, 18))  # Aantal teams is 18
    wedstrijden['SegmentUit'] = wedstrijden['PositieUit'].apply(lambda x: segment_bepalen(x, 18))
    
    # Categoriseer wedstrijden gebaseerd op segmenten
    conditions = [
        (wedstrijden['SegmentThuis'] == wedstrijden['SegmentUit']),
        ((wedstrijden['SegmentThuis'] == 1) | (wedstrijden['SegmentUit'] == 1)) & ((wedstrijden['SegmentThuis'] == 3) | (wedstrijden['SegmentUit'] == 3)),
        ((wedstrijden['SegmentThuis'] == 1) & (wedstrijden['SegmentUit'] == 2)) | ((wedstrijden['SegmentThuis'] == 2) & (wedstrijden['SegmentUit'] == 1))
    ]
    choices = ['dicht', 'gemiddeld', 'ver']
    wedstrijden['Categorie'] = np.select(conditions, choices, default='niet-geclassificeerd')
    
    return wedstrijden

# Functie om het totaal aantal wedstrijden en doelpunten per categorie te berekenen
def statistieken_berekenen(wedstrijden):
    aggregaties = wedstrijden.groupby('Categorie').agg(TotaalWedstrijden=('Categorie', 'size'), TotaalDoelpunten=('StandThuis', 'sum'))
    return aggregaties

def goodness_of_fit_test(aggregaties):
    # Berekenen van verwachte doelpunten per wedstrijd voor elke categorie
    aggregaties['DoelpuntenPerWedstrijd'] = aggregaties['TotaalDoelpunten'] / aggregaties['TotaalWedstrijden']
    
    # We maken een contingency table voor de chi-square test
    # Observed data
    observed = [
        [aggregaties.loc['dicht', 'TotaalDoelpunten'], aggregaties.loc['dicht', 'TotaalWedstrijden'] - aggregaties.loc['dicht', 'TotaalDoelpunten']],
        [aggregaties.loc['ver', 'TotaalDoelpunten'], aggregaties.loc['ver', 'TotaalWedstrijden'] - aggregaties.loc['ver', 'TotaalDoelpunten']]
    ]
    chi2, p, dof, expected = chi2_contingency(observed)
    
    return {'chi2_statistic': chi2, 'p_value': p, 'degrees_of_freedom': dof, 'expected_frequencies': expected}

# Voorbeeldgebruik
# Voeg eerst de functies en data samen zoals eerder beschreven
wedstrijden = voeg_posities_toe(wedstrijdendata, klassementdata)
wedstrijden = categoriseer_wedstrijden(wedstrijden)
aggregaties = statistieken_berekenen(wedstrijden)
test_resultaten = goodness_of_fit_test(aggregaties)

# Print de resultaten
print(test_resultaten)


ValueError: All values in `observed` must be nonnegative.